In [1]:
import os
import torch
import sys
sys.path.append("..")
from utils.misc import create_dir, create_log_dir, load_checkpoint, save_checkpoint, tab_printer, Params, set_logger
from utils.gelu import GELU
from generate.generate_images import generate_gif

In [2]:
import tensorflow as tf

In [10]:
torch.cuda.is_available()

True

In [11]:
torch.cuda.get_device_name(0)

'GeForce RTX 2070'

In [3]:
img = torch.randn(1, 1, 28, 28)

In [13]:
img.size(0)

1

In [ ]:
import torch.nn as nn

In [ ]:
def conv_out_shape(w, k, s, p=0):
    return (w - k + 2 * p) / s + 1

In [ ]:
def tconv_out_shape(w, k, s):
    return w * s + max(k - s, 0)

In [ ]:
tconv_out_shape(4, 3, 2)

In [ ]:
tconv_out_shape(9, 3, 2)

In [ ]:
tconv_out_shape(19, 2, 2)

In [ ]:
tconv_out_shape(27, 2, 1)

In [ ]:
[4, 3, 2]

In [ ]:
conv_out_shape(28, 4, 2)

In [ ]:
conv_out_shape(13, 3, 2)

In [ ]:
conv_out_shape(6, 3, 1)

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

In [ ]:
flatten = Flatten()

In [ ]:
f = flatten(x2)

In [ ]:
f.shape

In [ ]:
class UnFlatten(nn.Module):
    def __init__(self, channels, height, width):
        super(UnFlatten, self).__init__()
        self.channels = channels
        self.height = height
        self.width = width

    def forward(self, x):
        return x.view(x.size(0), self.channels, self.height, self.width)

In [ ]:
conv_layers = [
            nn.Conv2d(in_channels=1,
                      out_channels=64,
                      kernel_size=4,
                      stride=2),
            nn.BatchNorm2d(64),
            GELU(),
            nn.Conv2d(in_channels=64,
                      out_channels=128,
                      kernel_size=3,
                      stride=2),
            nn.BatchNorm2d(128),
            GELU(),
            nn.Conv2d(in_channels=128,
                      out_channels=256,
                      kernel_size=3,
                      stride=1),
            nn.BatchNorm2d(256),
            GELU(),
            Flatten(),
        ]

In [ ]:
conv_encoding = nn.Sequential(*conv_layers)

In [ ]:
i = conv_encoding(img)

In [ ]:
i.shape

In [ ]:
tconv_layers = [
            UnFlatten(256, 4, 4),
            nn.ConvTranspose2d(in_channels=256,
                               out_channels=128,
                               kernel_size=3,
                               stride=2),
            nn.BatchNorm2d(128),
            GELU(),
            nn.ConvTranspose2d(in_channels=128,
                               out_channels=64,
                               kernel_size=3,
                               stride=3),
            nn.BatchNorm2d(64),
            GELU(),
            nn.ConvTranspose2d(in_channels=64,
                               out_channels=1,
                               kernel_size=2,
                               stride=1),
            nn.Sigmoid(),
        ]

decoding = nn.Sequential(*tconv_layers)

In [ ]:
decoding(i).shape

In [ ]:
fc_mu = nn.Linear(in_features=4096, out_features=50)

In [ ]:
fc_mu(i)

In [ ]:
i.shape

In [ ]:
torch.Size([1, 4096])

In [ ]:
import gym

In [ ]:
torch.randn(torch.Size([1, 4096])).shape

In [ ]:
root = os.path.abspath('..')
model_dir = os.path.join(root, 'experiments/vae')
json_path = os.path.join(model_dir, 'params.json')

In [ ]:
json_path

In [ ]:
params = Params(json_path)

In [ ]:
params

In [ ]:
tab_printer(params)

In [4]:
generate_gif('/home/aktersnurra/Documents/Projects/variational-autoencoders/experiments/vae_w_bn/runs/2019-08-07_23-42-01/generated_images')